In [1]:
# NSE
import requests
import pandas as pd
import io

nse_url = "https://nsearchives.nseindia.com/content/equities/EQUITY_L.csv"

# create Session from 'real' browser
headers = {
    'User-Agent': 'Mozilla/5.0'
}

s = requests.Session()
s.headers.update(headers)

# do a get call now
url = 'https://nsearchives.nseindia.com/content/equities/EQUITY_L.csv'
r = s.get(nse_url)
s.close()

# saving it to pd df for further preprocessing
df_nse = pd.read_csv(io.BytesIO(r.content))
df_nse

,SYMBOL,NAME OF COMPANY,SERIES,DATE OF LISTING,PAID UP VALUE,MARKET LOT,ISIN NUMBER,FACE VALUE
0,20MICRONS,20 Microns Limited,BE,06-OCT-2008,5,1,INE144J01027,5
1,21STCENMGM,21st Century Management Services Limited,BE,03-MAY-1995,10,1,INE253B01015,10
2,360ONE,360 ONE WAM LIMITED,EQ,19-SEP-2019,1,1,INE466L01038,1
3,3IINFOLTD,3i Infotech Limited,EQ,22-OCT-2021,10,1,INE748C01038,10
4,3MINDIA,3M India Limited,EQ,13-AUG-2004,10,1,INE470A01017,10
...,...,...,...,...,...,...,...,...
2030,ZOTA,Zota Health Care LImited,EQ,19-AUG-2019,10,1,INE358U01012,10
2031,ZUARI,Zuari Agro Chemicals Limited,EQ,27-NOV-2012,10,1,INE840M01016,10
2032,ZUARIIND,ZUARI INDUSTRIES LIMITED,EQ,12-APR-1995,10,1,INE217A01012,10
2033,ZYDUSLIFE,Zydus Lifesciences Limited,EQ,18-APR-2000,1,1,INE010B01027,1


In [4]:
df_nse1 = df_nse[['SYMBOL','NAME OF COMPANY']]
df_nse1.to_csv('E:\\Projects\Finoobs\\Portfolio Optimization\\test\\indian_stock_listings.csv',index=False)

In [8]:
from splinter import Browser
from selenium import webdriver
import time, os
import pandas as pd

download_dir = "E:\\Projects\\Finoobs\\tmp"
bse_link = "https://mock.bseindia.com/corporates/List_Scrips.html"

# Set download preferences
prefs = {"download.default_directory": download_dir}
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", prefs)

# Initialize browser
try:
    browser = Browser('chrome', options=options)
    
    # Visit link
    browser.visit(bse_link)

    # Fill out form fields
    browser.find_by_id('ddlsegment').select("Equity")
    browser.find_by_id('ddlstatus').select("Active")

    # Hit submit button
    browser.find_by_id('btnSubmit').click()

    # Let the table load
    browser.is_element_present_by_text("Issuer Name", wait_time=10)

    # Download
    browser.find_by_id('lnkDownload').click()

    # Wait for the file to download
    file_path = os.path.join(download_dir, "Equity.csv")
    wait_time = 0
    while not os.path.exists(file_path) and wait_time < 30:  # wait max 30 seconds
        time.sleep(1)
        wait_time += 1

    if os.path.exists(file_path):
        # Load downloaded file
        df_bse = pd.read_csv(file_path)
        print(df_bse)
    else:
        print("Download failed or file not found.")
        
finally:
    browser.quit()


      Security Code                            Issuer Name Security Id  \
0            500002                      ABB India Limited         ABB   
1            500003                   Aegis Logistics Ltd.    AEGISLOG   
2            500008   Amara Raja Energy & Mobility Limited       ARE&M   
3            500009       Ambalal Sarabhai Enterprise Ltd.   AMBALALSA   
4            500012             Andhra Petrochemicals Ltd.   ANDHRAPET   
...             ...                                    ...         ...   
4399         890197                    Yarn Syndicate Ltd.      YARNPP   
4400         890200              Iykot Hitech Toolroom Ltd     IYKOTPP   
4401         890202  SOLARA ACTIVE PHARMA SCIENCES LIMITED    SOLARAPP   
4402         890205                          Sobha Limited     SOBHAPP   
4403         890207                 Sharat Industries Ltd.    SHARATPP   

                          Security Name  Status Group  Face Value  \
0                     ABB India Limited  A

In [17]:
# Merge the list 
# some columns naming convention was different
df_bse = df_bse.rename(columns={
    "Security Name": "NAME OF COMPANY",
    "Security Id": "SYMBOL"
})
df_bse1 = df_bse[['SYMBOL','NAME OF COMPANY']]
# df_bse1['NAME OF COMPANY'] = df_bse1['NAME OF COMPANY'].str.title()
df_nse1 = df_nse[['SYMBOL','NAME OF COMPANY']]
df_bse1.loc[:, 'NAME OF COMPANY'] = df_bse1['NAME OF COMPANY'].str.title()
df_nse1.loc[:, 'NAME OF COMPANY'] = df_nse1['NAME OF COMPANY'].str.title()
# merged on SYMBOL
# final_df = pd.merge(df_nse1, df_bse1, on='SYMBOL',  how="outer")
# final_df
# Merge on SYMBOL only
final_df = pd.merge(df_nse1, df_bse1, on='SYMBOL', how="outer", suffixes=('_NSE', '_BSE'))

# Combine NAME OF COMPANY columns, preferring NSE if available
final_df['NAME OF COMPANY'] = final_df['NAME OF COMPANY_NSE'].combine_first(final_df['NAME OF COMPANY_BSE'])

# Drop the extra columns
final_df = final_df[['SYMBOL', 'NAME OF COMPANY']]

# Display final merged DataFrame
final_df

,SYMBOL,NAME OF COMPANY
0,08ABB,Nippon India Equity Savings Fund - Segregated...
1,08ADD,Nippon India Equity Savings Fund - Segregated...
2,08ADR,Nippon India Equity Savings Fund - Segregated...
3,08AGG,Nippon India Equity Savings Fund - Segregated...
4,08AMD,Nippon India Equity Savings Fund - Segregated...
...,...,...
4529,ZUARI,Zuari Agro Chemicals Limited
4530,ZUARIIND,Zuari Industries Limited
4531,ZWELCAST,Welcast Steels Ltd.
4532,ZYDUSLIFE,Zydus Lifesciences Limited


In [ ]:
# download all the data
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Function to download stock data
def download_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    # if 'Adj Close' in stock_data.columns:
    #     return stock_data['Adj Close']
    return stock_data


# Set the date range for the last 10 years
end_date = datetime.now().date()
start_date = end_date - timedelta(days=10*365)

# Download data for each ticker and save it in a separate CSV file
tickers = df_nse['SYMBOL']

for ticker in tickers:
    print(f"Downloading data for {ticker}...")
    data = download_stock_data(ticker+'.NS', start_date, end_date)
    if data is not None:
        # Save to CSV file
        # csv_filename = f"{ticker.replace('.NS', '')}_adj_close_last_10_years.csv"
        data.to_csv('/workspaces/Finoobs/Portfolio Optimization/stocks/'+ticker+'.csv')
        print(f"Saved data to {ticker}")

print("Data download and saving complete.")

In [ ]:
df_nse['SYMBOL'][:10]